# Random forests P22250 model

## Setup

### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
import os

## Data

Get PUF records.

In [2]:
train = pd.read_csv('puf80%training')
test = pd.read_csv('puf20%testing')

cols =  ['P22250', 'P22250_sign','DSI', 'EIC', 'MARS', 'XTOT', 'E00200', 
         'E00300', 'E00400', 'E00600', 'E00650', 'E00800', 'E00900', 'E01100', 
         'E01400', 'E01500', 'E01700', 'E02100', 'E02300', 'E02400', 'E03150', 
         'E03210', 'E03240', 'E03270', 'E03300', 'E17500', 'E18400', 'E18500', 
         'E19200', 'E19800', 'E20100', 'E20400', 'E32800', 'F2441', 'N24']

predictors = cols[2:]

train = train[cols]
test = test[cols]

In [3]:
X_train = train[predictors]
X_test = test[predictors]

Y_train = train['P22250']
Y_test = test['P22250']

Y_train_sign = train['P22250_sign']
Y_test_sign = test['P22250_sign']

## Model

Train a random forests model.

In [4]:
# 100 estimators
N_ESTIMATORS = 100
rf = ensemble.RandomForestRegressor(n_estimators=N_ESTIMATORS, 
                                    min_samples_leaf=1, random_state=3, 
                                    verbose=True, 
                                    n_jobs=-1)  # Use maximum number of cores.
rf.fit(X_train, Y_train)

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.0min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=3, verbose=True, warm_start=False)

### Model description

Show the ten most important features.

In [5]:
feature_importance = pd.Series(rf.feature_importances_, index=X_train.columns)
feature_importance.sort_values(ascending=False)[:10]

E00300    0.213896
E00200    0.088474
E00600    0.077358
E18400    0.066015
E19200    0.065496
MARS      0.058366
E20400    0.044849
XTOT      0.043097
E18500    0.040825
E00650    0.039985
dtype: float64

## Predict

### Top-line (average)

In [6]:
pred = pd.DataFrame({'actual': Y_test,
                     'pred': rf.predict(X_test)})
pred['error'] = pred.pred - pred.actual
pred['actual_sign'] = np.sign(pred.actual)
pred['pred_sign'] = np.sign(pred.pred)
pred['correct_sign'] = (pred.actual_sign == pred.pred_sign)
pred['count'] = 1

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.6s finished


MAE, RMSE, and % negative/zero/positive.

In [7]:
pred.error.abs().mean()

35169.69998756283

In [8]:
pred.error.pow(2).mean() ** 0.5

294425.26134793647

In [9]:
pred.pivot_table(index='actual_sign', columns='pred_sign', values='count', 
                 aggfunc=sum, margins=True)

pred_sign,-1.0,0.0,1.0,All
actual_sign,,,,
-1.0,3078,54,1543,4675
0.0,7626,11197,5161,23984
1.0,2465,39,1450,3954
All,13169,11290,8154,32613


In [10]:
pred.correct_sign.mean()

0.48216968693465795

### All trees

In [11]:
preds = []
for estimator in rf.estimators_:
    preds.append(estimator.predict(X_test))
preds = np.array(preds).transpose()  # One row per record.

In [12]:
rand_col = np.random.randint(N_ESTIMATORS, size=preds.shape[0])
random_tree = preds[np.arange(preds.shape[0]), rand_col]

In [13]:
pred_random_tree = pd.DataFrame({'actual': Y_test,
                                 'pred': random_tree})
pred_random_tree['error'] = pred_random_tree.pred - pred_random_tree.actual
pred_random_tree['actual_sign'] = np.sign(pred_random_tree.actual)
pred_random_tree['pred_sign'] = np.sign(pred_random_tree.pred)
pred_random_tree['correct_sign'] = (
    pred_random_tree.actual_sign == pred_random_tree.pred_sign)
pred_random_tree['count'] = 1

As expected, MAE and RMSE exceed values from the point estimate prediction.

In [14]:
pred_random_tree.error.abs().mean()

48934.12760859037

In [15]:
pred_random_tree.error.pow(2).mean() ** 0.5

550189.4457893613

But the distribution of sign is closer to correct, since it's not averaging out the zeros.

In [16]:
pred_random_tree.pivot_table(index='actual_sign', columns='pred_sign', 
                             values='count', aggfunc=sum, margins=True)

pred_sign,-1.0,0.0,1.0,All
actual_sign,,,,
-1.0,1859,1359,1457,4675
0.0,1635,20856,1493,23984
1.0,1527,1285,1142,3954
All,5021,23500,4092,32613


In [17]:
pred_random_tree.correct_sign.mean()

0.7315181062766382

#### Log-loss of sign

In [18]:
preds_neg = np.sum(preds < 0, axis=1) / 100
preds_zero = np.sum(preds == 0, axis=1) / 100
preds_pos = np.sum(preds > 0, axis=1) / 100

rf_pred_proba = list(map(list, zip(*[preds_neg, preds_zero, preds_pos])))

metrics.log_loss(Y_test_sign, rf_pred_proba)

0.6065213242184263

## Multinomial logistic regression of sign

In [19]:
mult = linear_model.LogisticRegression(
    multi_class='multinomial', solver='newton-cg', random_state=3)
mult.fit(X_train, Y_train_sign)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=3, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

### Coefficients

p-values are tricky to calculate, so just show the features with highest absolute coefficient on the zero class.

In [20]:
mult_coef = pd.DataFrame({
    'negative': mult.coef_[0],
    'zero': mult.coef_[1],
    'positive': mult.coef_[2],
    'abs_zero': np.abs(mult.coef_[1])},
    index=X_train.columns)

mult_coef.sort_values('abs_zero', ascending=False)[:10]

,abs_zero,negative,positive,zero
XTOT,0.052191,-0.025528,-0.026663,0.052191
MARS,0.051600,-0.025386,-0.026214,0.051600
E01100,0.015469,-0.007879,-0.007590,0.015469
N24,0.013334,-0.006632,-0.006702,0.013334
EIC,0.007890,-0.003979,-0.003911,0.007890
E03210,0.002384,-0.001416,-0.000968,0.002384
DSI,0.002189,-0.001100,-0.001089,0.002189
F2441,0.000403,-0.000208,-0.000194,0.000403
E32800,0.000232,-0.000098,-0.000135,0.000232
E02300,0.000145,-0.000058,-0.000086,0.000145


### Predict

In [21]:
mult_pred = pd.DataFrame({'actual': Y_test_sign,
                          'pred': mult.predict(X_test)})
mult_pred['sign_correct'] = (mult_pred.actual == mult_pred.pred)
mult_pred['count'] = 1
mult_pred.sign_correct.mean()

0.7756722779259805

In [22]:
mult_pred.pivot_table(index='actual', columns='pred', 
                      values='count', aggfunc=sum, margins=True)

pred,-1,0,1,All
actual,,,,
-1,1875,2671,129,4675
0,629,23291,64,23984
1,1364,2459,131,3954
All,3868,28421,324,32613


#### Log-loss

In [23]:
mult_pred_proba = mult.predict_proba(X_test)
metrics.log_loss(Y_test_sign, mult_pred_proba)

0.8714288788771535